## Intro til geopandas for GIS-folk

Åpne heller notebook-en her:

https://nbviewer.org/github/statisticsnorway/intro_til_geopandas/blob/main/3_geopandas_for_GIS_folk.ipynb

Notebook-en er todelt:
- første halvdel: pandas/Python
- andre halvdel: geopandas/GIS

* [Dissolve](#first-bullet)

### Aller først: hva er geopandas?

Geopandas er en python-pakke ala arcpy og pyqgis.

GIS-biten er basert på de samme algoritmene som i QGIS, POSTGIS osv. (fra GEOS, GDAL og PROJ).

Det som ikke har med GIS å gjøre er basert på pakken pandas.

Første del av denne notebooken handler mest om pandas. Andre del om geopandas.

### Så hvordan fungerer pandas (og Python)?

#### Tall:

In [ ]:
tall = 6
tall

#### Tekst:

In [ ]:
tekst = "0301"
tekst

#### Liste
Lister som inneholder tall eller tekst. Eller andre ting (alt er lov å putte inni lister)

In [ ]:
liste = [tekst, "3401", "3401", "0301", "3401"]
liste

In [ ]:
liste2 = [tall, 3, 8, 24, 11]
liste2

#### tuple:
Nesten som lister, men litt mindre fleksible fordi de ikke kan endres på (men de tar mindre plass i minnet)

In [ ]:
tuple1 = (tall, 3, 8, 24, 11)
tuple1

#### Ordbok (dictionary)

De består av nøkkel-verdi-par (key-value pairs)
- verdier, som kan være hva som helst, for eksempel en liste
- nøkler som er knyttet til verdiene

In [ ]:
verdier = [1,2,3]
ordbok = {"nøkkel": verdier}
ordbok

In [ ]:
ordbok = {"KOMMUNENR": liste, "tall": liste2}
ordbok

In [ ]:
ordbok["KOMMUNENR"]

In [ ]:
ordbok["tall"]

Dette er de viktigste innebygde objekttypene i Python.

En Pandas DataFrame er en litt mer fancy objekttype, men den er basert på blant annet de innebygde Python-typene.

Hver objekttype i Python har spesifikke metoder/funksjoner knyttet til seg. Syntaxen er gjerne sånn her:

objekt.metode()

For eksempel har tekst-typen (string) metoder som lower og zfill:

In [ ]:
tekst = "KOMMUNENR"
tekst.lower() # til små bokstaver

In [ ]:
tekst = "301"
tekst.zfill(4)  #fyller på med ledende 0 hvis færre enn fire bokstaver

Lister har egne metoder:

In [ ]:
liste = [1, 2, 3]
liste.remove(2) # fjerner verdien 2 fra lista (fjerner bare én hvis flere 2-ere)
liste.reverse() # snur lista
liste

### pandas

In [ ]:
import pandas as pd

En pandas.DataFrame er en slags ordbok bestående av nøkler (kolonnenavn) og verdier (kolonneverdier):

In [ ]:
pd.DataFrame(ordbok)

Altså:

In [ ]:
df = pd.DataFrame({
    "KOMMUNENR": ['0301', "3401", "3401", "0301", "3401"],
    "tall": [6, 3, 8, 24, 11]
})
df

DataFramen har egne metoder knyttet til seg, for eksempel:

In [ ]:
df.drop_duplicates("KOMMUNENR") # beholder første verdi for hvert kommunenummer

In [ ]:
df.groupby("KOMMUNENR").sum() # summerer for hvert unike kommunenummer

Mer om det under.

Man refererer til en kolonne på samme måte som nøklene i en ordbok:

In [ ]:
ordbok["tall"]

In [ ]:
df["tall"]

Man kan endre hver rad i kolonnen direkte (trenger ingen field calculator):

In [ ]:
df["tall"] + 100

For å lagre det som en ny kolonne:

In [ ]:
df["tall_pluss_100"] = df["tall"] + 100
df

Man kan også referere til kolonner med punktum (det funker riktignok ikke å lage en ny kolonne på denne måten):

In [ ]:
df.tall

### geopandas
Samme regler gjelder for geopandas.

In [ ]:
import geopandas as gpd

I geopandas starter man med å lese inn dataene sine som en GeoDataFrame.

For eksempel vegdata for Oslo:

In [ ]:
veger = gpd.read_file("C:/users/ort/namibia/roads_oslo_2020.gpkg")
veger

Legg merke til kolonnen "geometry". Den forteller at vi har linjegeometrier, og koordinatene ser ut til å være i utm-format.

Det stemmer:

In [ ]:
veger.crs

Når man leser filen, kan man også velge hvilke kolonner og rader man vil lese. 

Og fjerne Z-koordinater.

In [ ]:
import pyogrio
noen_veger = gpd.read_file("C:/users/ort/namibia/roads_oslo_2020.gpkg", engine = "pyogrio",
                       
                       columns = ["speed_limit", "roadtype"],
                      
                       where = "speed_limit >= 80 and roadtype in ('R', 'E')", # OBS: syntaxen er litt annerledes for lesing av geoparquet-filer
                           
                       force_2d = True,
                     )

print(noen_veger.speed_limit.value_counts())
print(noen_veger.roadtype.value_counts())

noen_veger.plot()
noen_veger

Man kan også lese kun et område, for eksempel 500 meter rundt SSB i Akersveien:

In [ ]:
from shapely.wkt import loads
akersveien = gpd.GeoDataFrame({"geometry": gpd.GeoSeries(loads("POINT (10.7476913 59.9222196)"))}, crs=4326).to_crs(25833)
akersveien["geometry"] = akersveien.buffer(500)

veger = gpd.read_file("C:/users/ort/namibia/roads_oslo_2020.gpkg", 
                      engine = "pyogrio",
                      bbox = akersveien
                     )
veger.explore()

Men nå vil jeg ha alle veger:

In [ ]:
veger = gpd.read_file("C:/users/ort/namibia/roads_oslo_2020.gpkg")
veger.plot()

For å kartlegge, kan man bruke plot for statiske kart, og explore for interaktive. Det statiske kartet bruker en python-pakke for graflegging (matplotlib), derfor får man koordinatene i x- og y-aksene:

In [ ]:
veger.plot("traffic_per_day", scheme="NaturalBreaks")

Man kan også undersøke dataene grafisk:

In [ ]:
veger["traffic_per_day"].plot.hist()

Eller se på snitt, sum, median osv:

In [ ]:
veger["traffic_per_day"].describe()

### Syntaksen i pandas og geopandas er som regel enten:

- data.attributt

- data.metode()

- data.kolonne.attributt

- data.kolonne.metode()

En attributt inneholder informasjon om dataene.

En metode innebærer å gjøre endringer i dataene. 

En geopandas.GeoDataFrame har alle attributter og metoder som tilhører pandas.DataFrame, og i tillegg en del GIS-attributter og -metoder.

Her er noen (Geo)DataFrame-attributter:

In [ ]:
veger.columns # liste over kolonnene

In [ ]:
veger.shape # antall rader og kolonner

In [ ]:
veger.crs # koordinatsystemet

In [ ]:
veger.traffic_per_day # kolonnene regnes også som attributter

In [ ]:
veger.area # = Shape_Area

In [ ]:
veger.length # = Shape_Leng

Man kan gjøre matte direkte på kolonner og area/length-attributtene:

In [ ]:
veger["traffic_per_km"] = veger.traffic_per_day / (veger.length / 1000)
veger.head()

Med tekstkolonner kan man bruke pandas sin str-attributt, som gjør at man kan bruke metoder som tilhører Pythons innebygde string-type (altså tekst), og noen metoder til.

In [ ]:
veger["FYLKE"] = veger.municipality.str[:2] # velg ut to første tegnene
veger["roadtype"] = veger.roadtype.str.lower() # små bokstaver
veger["municipality"] = veger.municipality.str.zfill(4) # legg til ledende 0 i 3-sifrede kommunenumre
veger.head()

Her gjøre man altså dette for hver rad i kolonnen:

In [ ]:
"0301"[:2] # velg ut to første tegnene

In [ ]:
"K".lower() # små bokstaver

In [ ]:
"0301".zfill(4) # legg til ledende 0 i 3-sifrede kommunenumre

GIS-funksjoner er metoder som tilhører geodataframen. Så man skriver gjerne:

data.gisfunksjon()

In [ ]:
veger.buffer(10)

In [ ]:
veger.dissolve(by="roadtype", aggfunc="sum") # med by='roadtype' holder man vegkategoriene adskilt, og man får summen av de andre kolonnene for hver vegkategori

geopandas.dissolve er basert på pandas groupby + agg. Denne gjør samme grupperte oppsummering, men samler ikke geometrien:

In [ ]:
veger.groupby("roadtype").sum(numeric_only = True)

Hvis man vil ha ulike aggregeringsmetoder:

In [ ]:
summary_statistics = veger.groupby("roadtype").agg(
    
    traffic_per_day_sum = ('traffic_per_day','sum'), # kolonnenavn og aggregeringsmetode i tuple, altså med vanlig parentes, adskilt med komma
    
    speed_limit_mean = ('speed_limit','mean'),
    
    municipality = ('municipality','first')

)
summary_statistics

## Velg kolonner

In [ ]:
kolonner_du_vil_beholde = ["traffic_per_day", "traffic_per_km", "geometry"]

veger2 = veger[kolonner_du_vil_beholde]

veger2.head()

Samme som over, men på én linje:

In [ ]:
veger2 = veger[["traffic_per_day", "traffic_per_km", "geometry"]]
veger2.head()

Man kan fjerne kolonner med drop():

In [ ]:
veger2 = veger.drop(["speed_limit", "tunnel", "bridge", "roadtype", "municipality"], 
                    axis=1) # axis=1 betyr at man vil fjerne kolonner. axis=0 betyr rader
veger2.head()

## Velg rader
Altså som select i arcpy.

Sånn fjerner man rader med trafikk under 1000:

In [ ]:
veger2 = veger.loc[veger.traffic_per_day > 1000]

veger2.traffic_per_day.min()

Man spør Python om kolonneverdiene er større enn 1000

In [ ]:
2000 > 1000

In [ ]:
500 > 1000

Dette gjøres for hver rad:

In [ ]:
veger.traffic_per_day > 1000

Og så velger man ut radene som er True

In [ ]:
veger.loc[veger.traffic_per_day > 1000]

Man kan også velge rader med pandas.query. Her trenger man bare å skrive kolonnenavnene (ikke tabellnavnet). Tar dermed mindre plass hvis man har mange kondisjoner:

In [ ]:
veger2 = veger.query("traffic_per_day > 1000 | roadtype == 'E' & tunnel == 0")

## Velg rader og kolonner samtidig

Inni loc kan man velge først rader, så et komma, så kan man velge kolonner:

In [ ]:
veger2 = veger.loc[veger.traffic_per_day > 1000, ["traffic_per_day", "traffic_per_km", "geometry"]]
veger2

Behold bare rader med trafikk over 1000 eller europaveger som ikke er tunneller.

Og behold bare kolonner som inneholder ordene 'traffic'. Og behold geometri-kolonnen.

In [ ]:
veger2 = veger.loc[(veger.traffic_per_day > 1000) | (veger.roadtype == 'E') & (veger.tunnel == 0),                    
                   (veger.columns.str.contains('traffic')) | (veger.columns=="geometry")]
veger2

Fjern kolonner som inneholder ordene 'traffic' eller 'speed' (og behold alle rader):

In [ ]:
veger2 = veger.loc[:, ~veger.columns.str.contains('traffic|speed')]
veger2

Hvis man plasserer loc-opplegget på motsatt side av likhetstegnet, kan man endre kolonneverdier for kun radene man velger.

Her endrer man trafikk-kolonnen til 1000 for alle rader med trafikkmengde over 1000:

In [ ]:
veger2 = veger.copy() # kopi for å ikke ødelegge de opprinnelige dataene. Men man bør egentlig unngå å kopiere for mye, fordi man da bruker mer minne.

veger2.loc[veger2.traffic_per_day > 1000, "traffic_per_day"] = 1000

Nå har man ikke fjernet noen rader, men man har gitt alle de mest trafikkerte vegene (ÅDT>1000) en ÅDT på 1000:

In [ ]:
veger2

## Table join

Hvis man bare trenger én eller få kolonner fra et annet datasett, er map effektivt og raskt. Man kan plassere (blant anent) en pandas-kolonne inni map:

In [ ]:
aggregert = veger.groupby('roadtype')['traffic_per_day'].mean()

veger["traffic_roadtype_mean"] = veger["roadtype"].map(aggregert)

veger

Dette funker fordi man matcher verdiene i kolonnen veger.roadtype med index-en 'roadtype' i de aggregerte dataene. Dette gjøres for hver rad:

In [ ]:
aggregert["k"] # små bokstaver fordi jeg endra det lenger oppe

In [ ]:
aggregert["r"]

Man kan også gjenta dette for flere kolonner. Da venter man med å hente ut kolonnen til inni map:

In [ ]:
aggregert = veger.groupby('roadtype').mean(numeric_only=True) # dette er nå en dataframe

veger["traffic_roadtype_mean"] = veger["roadtype"].map(aggregert['traffic_per_day']) # henter ut en Series inni map
veger["speed_roadtype_mean"] = veger["roadtype"].map(aggregert['speed_limit'])

veger

Joining av hele tabeller gjøres med pandas.merge (OBS: tilsvarer ikke arcpy.merge)

La oss si vi vil ha kommunearealet som en kolonne i vegdataene våre. Henter først kommunedata.

In [ ]:
kommuner = gpd.read_file(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\data\Basisdata_0000_Norge_25833_Kommuner_FGDB.gdb", layer="kommune", engine="pyogrio",
                        columns = ["kommunenummer", "geometry"],
                        ).rename(columns={"kommunenummer":"KOMMUNENR"})
kommuner["kommuneareal"] = kommuner.area
komm_areal = kommuner[["KOMMUNENR", "kommuneareal"]]

Så kan det kobles:

In [ ]:
joinet = veger.merge(komm_areal, 
                     left_on = "municipality", 
                     right_on = "KOMMUNENR")
joinet.head()

Hvis join-kolonnene har samme navn i begge tabeller, bruker man parameteret 'on':

In [ ]:
joinet = (veger
          .rename(columns={"municipality":"KOMMUNENR"})
          .merge(komm_areal, 
                 on = "KOMMUNENR")
          )
joinet.head()

Som default gjøres en inner join, som vil si at rader uten matchende verdier forsvinner.

Man kan også gjøre blant annet left, right og outer join:

In [ ]:
for how in ["inner", "left", "right", "outer"]:
    
    joinet = veger.rename(columns={"municipality":"KOMMUNENR"}).merge(komm_areal, on = "KOMMUNENR",
                    how = how) 
    
    print(f"antall rader {how}-join: {len(joinet)}")

## pandas.concat

Samler tabeller basert på kolonnenavn.

Som merge i arcpy.

La oss si kommunedataene er spredt i to tabeller:

In [ ]:
tabell1 = kommuner.copy().loc[kommuner.KOMMUNENR.astype(int) < 3500] # tar her en kopi for å unngå advarselen om kopi kontra view. Vil nok ikke være nødvendig i framtiden.
tabell2 = kommuner.copy().loc[kommuner.KOMMUNENR.astype(int) >= 3500]
tabell1["fra_hvilken_tabell"] = "tabell1"
tabell2["fra_hvilken_tabell"] = "tabell2"
tabell2["kolonne_kun_i_tabell2"] = 1 # her får man manglende verdier for radene fra tabell 1

Da kan de samles sånn her:

In [ ]:
samlet = pd.concat([tabell1, tabell2], ignore_index=True)
samlet

## Buffer
geopandas.buffer returnerer bare geometrikolonnen:

In [ ]:
veger.buffer(500)

Oppløsningen på bufringen er også lavere enn i ArcGIS, har Erik oppdaget.

For å beholde de andre kolonnene, ha høyere buffer-oppløsning, reparere geometri og ikke overskrive 'veger'-objektet, kan man gjøre dette:

In [ ]:
veger_bufret = veger.copy()

veger_bufret["geometry"] = veger_bufret.buffer(500, resolution = 50)

veger_bufret["geometry"] = veger_bufret.make_valid()

veger_bufret

Jeg vil ikke skrive alt det der hver gang. Derfor lager jeg en funksjon:

In [ ]:
def buff(gdf, avstand, resolution=50, copy=True, **qwargs): # **qwargs betyr at man tillater flere parametre. I dette tilfellet alle parametrene som godtas av geopandas' buffer()
    
    if copy:
        gdf2 = gdf.copy()
    else:
        gdf2 = gdf
    
    gdf2["geometry"] = gdf2.buffer(avstand, resolution=resolution, **qwargs)
    gdf2["geometry"] = gdf2.make_valid()
    
    return gdf2

In [ ]:
veger_bufret = buff(veger, 500)
veger_bufret

OBS: å ta kopier gjør at man bruker mer minne. Hvis man ikke trenger å beholde det ikke-bufrede objektet, kan det være greit å ikke ta kopi.

## Spatial join

La oss si vi har punktdata på skoler i Norge uten info om kommunenummer:

In [ ]:
skoler = gpd.read_file(r"C:\python\Befolkning_0000_Norge_25833_Grunnskoler_FGDB.gdb", layer="Grunnskole")
skoler = skoler[["skolenavn", "geometry"]]
skoler

Da kan vi gjøre en romlig kobling med kommunedata med geopandas.sjoin.

Denne fungerer på samme måte som pandas.merge, bare at man kobler basert på geometrien.

In [ ]:
# her returneres skolepunktene med info om kommunenummer
skoler.sjoin(kommuner)

Resultatene har én rad mindre enn skoledataene hadde i utgangspunktet. Det er fordi én skole ikke overlappet med noen kommuner. Denne ble fjernet fordi sjoin gjør en inner join som default. 

Man kan endre til left join for å få med skolen som ikke overlapper med kommunene.

La oss se hvor denne ene skolen ligger:

In [ ]:
(skoler
 .sjoin(kommuner, how="left")
 .query("KOMMUNENR.isna()") # velg ut bare skolen som mangler kommunenummer for å kartlegge
 .explore()
)

Med motsatt rekkefølge i sjoin, får man kommunene som inneholder en skole:

In [ ]:
kommuner.sjoin(skoler).plot()

Man får én rad per skole i kommunen. 

For Kongsvinger kommune, får man 11 rader med samme geometri (kommuneflaten), men ulike skolenavn:

In [ ]:
kommuner.sjoin(skoler).loc[kommuner.KOMMUNENR=="3401"].drop("index_right", axis=1)

Det er flest skoler i Oslo:

In [ ]:
# tell opp antall rader av hvert kommunenummer, det vil si antall skoler per kommune
kommuner.sjoin(skoler).KOMMUNENR.value_counts()

sjoin_nearest kan brukes for å finne avstand til nærmeste skole, og samtidig få kolonneinfoen, i dette tilfellet navnet på nærmeste skole:

In [ ]:
tilfeldige_punkter = veger.sample(1000)
tilfeldige_punkter["geometry"] = tilfeldige_punkter.centroid

joinet = tilfeldige_punkter.sjoin_nearest(skoler, 
                                          distance_col = "meter_til_skole").drop("index_right", axis=1)

joinet.plot("meter_til_skole", scheme="Quantiles", legend=True)
joinet.sample(3)

Hvis man vil lage en kategorisk kolonne med ja/nei hvis mindre/mer enn 500 meter til nærmeste skole:

In [ ]:
import numpy as np # pakken pandas er basert på

In [ ]:
joinet["skole_innen_500m"] = np.where(joinet["meter_til_skole"] < 500, 
                                     "ja",
                                     "nei")

joinet.plot("skole_innen_500m", cmap="plasma", legend=True)

## Overlay

Geopandas har en overlay-funksjon som gjør alt.

Geometrien repareres også automatisk.

Man spesifiserer hvilken overlay man vil ha med 'how'-argumentet. Mer detaljer om det i notebook-en om vernede områder.

Velger noen tilfeldige punkter og bufrer dem 500 meter:

In [ ]:
bufrede_punkter = tilfeldige_punkter.sample(100)
bufrede_punkter["geometry"] = bufrede_punkter.buffer(500)
bufrede_punkter.plot()

Her er de fem overlay-mulighetene:

In [ ]:
overlay1 = veger.overlay(bufrede_punkter, how = "intersection", keep_geom_type=True)
overlay1.plot()

overlay2 = veger.overlay(bufrede_punkter, how =  "difference", keep_geom_type=True) # = erase i ArcGIS
overlay2.plot()

overlay3 = veger.overlay(bufrede_punkter, how =  "symmetric_difference", keep_geom_type=True) # returnerer det som ikke overlapper i begge dataene
overlay3.plot()

overlay4 = veger.overlay(bufrede_punkter, how =  "identity", keep_geom_type=True)
overlay4.plot()

overlay5 = veger.overlay(bufrede_punkter, how =  "union", keep_geom_type=True)
overlay5.plot()

## Dissolve og til singlepart <a class="anchor" id="first-bullet"></a>

Dette samler alle radene:

In [ ]:
dissolvet = bufrede_punkter.dissolve()
dissolvet

Dette deler opp ikke-overlappende, altså fra multipolygon til polygon (multipart to singlepart):

In [ ]:
singlepart = dissolvet.explode()

singlepart.explore()

Man kan dissolve 'by' kolonner og velge aggregeringsmetode (default er 'first', som er uheldig):

In [ ]:
dissolvet = bufrede_punkter.dissolve(
    by = ["tunnel", "bridge"],
    aggfunc = "sum" # first er default aggregeringsfunksjon
    ) 
dissolvet

Dette er det samme som pandas groupby + agg, bare at man da ikke får geometrien. 

Dette er som summary statistics i ArcGIS.

In [ ]:
oppsummert_uten_geometri = (bufrede_punkter
                            .groupby(["tunnel", "bridge"])
                            .agg("sum")
            )
oppsummert_uten_geometri

#### Kort om index og feilmeldingene det fører til

Index er et viktig konsept i pandas, men også i Python generelt.

Index-en starter på 0

In [ ]:
liste = [1,2,3]
liste[0]

I pandas har man denne numeriske index-en som standard, men man kan også velge hvilke kolonner som skal være index.

Det gjør noen ting enklere å jobbe med (mer om der her: https://www.youtube.com/watch?v=OYZNk7Z9s6I),

men ofte kan index-en skaper problemer og unødvendige feilmeidldinger 

For eksempel når man dissolver 'by' eller bruker pandas.groupby, blir kolonnene man grupperer etter til index for dataframen.

Nå som vi dissolvet 'by' tunnel og bridge, blir disse to til en MultiIndex:

In [ ]:
dissolvet.index

Og tunnel og brigde er ikke lenger kolonner:

In [ ]:
dissolvet.columns

Vi kan få dem tilbake som kolonner med reset_index:

In [ ]:
dissolvet = dissolvet.reset_index()
dissolvet

Hvis vi gjør det igjen nå som index er nullstilt, får vi en kolonne kalt "index":

In [ ]:
dissolvet = dissolvet.reset_index()
dissolvet

Og hvis enda en gang, får vi en index-kolonne for index-nivå 0:

In [ ]:
dissolvet = dissolvet.reset_index()
dissolvet

Og hvis enda en reset_index, gir pandas opp fordi kolonnen "level_0" allerede finnes...

In [ ]:
dissolvet = dissolvet.reset_index()
dissolvet

Vi kan fortsatt resette index hvis vi dropper å få index-en som kolonne:

In [ ]:
dissolvet = dissolvet.reset_index(drop=True)
dissolvet

Men hvis vi hadde satt drop=True til å begynne med, altså rett etter dissolve eller groupby().agg(), ville vi mistet tunnel- og bridge-kolonnene for godt. 

Før vi går videre, må alle disse index-kolonnene vekk:

In [ ]:
dissolvet = dissolvet.loc[:, ~dissolvet.columns.str.contains("index|level_")]
dissolvet

Index skaper også problemer i spatial join, siden denne, tåpelig nok, returnerer en kolonne kalt "index_right":

In [ ]:
dissolvet = dissolvet.sjoin(bufrede_punkter[["geometry"]])
dissolvet

Som gir feilmelding neste gang fordi "index_right" ikke kan være kolonnenavn i en join:

In [ ]:
dissolvet = dissolvet.sjoin(bufrede_punkter[["geometry"]])
dissolvet

### Heldigvis kan man unngå alle disse problemene ved å lage sine egne funksjoner:

In [ ]:
def diss(gdf, **qwargs):
    
    dissolvet = gdf.dissolve(**qwargs)
    
    dissolvet = dissolvet.reset_index()
    
    # kolonner fra tuple til string
    dissolvet.columns = ["_".join(kolonne).strip("_") if isinstance(kolonne, tuple) else kolonne for kolonne in dissolvet.columns]

    # fjern index-kolonner
    dissolvet = dissolvet.loc[:, ~dissolvet.columns.str.contains('index|level_')]

    return dissolvet


def min_sjoin(gdf_left, gdf_right, **qwargs):
    
    # fjern index-kolonner
    left_kopi = gdf_left.loc[:, ~gdf_left.columns.str.contains('index|level_')]
    right_kopi = gdf_right.loc[:, ~gdf_right.columns.str.contains('index|level_')]
    
    joinet = left_kopi.sjoin(right_kopi, **qwargs)

    # fjern index-kolonner igjen
    joinet = joinet.loc[:, ~joinet.columns.str.contains('index|level_')]
    
    return joinet

Nå funker ting som det skal uten å måtte skrive/kopiere så mye tekst hver gang:

In [ ]:
dissolvet = diss(bufrede_punkter[["tunnel", "bridge", "traffic_per_day", "speed_limit", "geometry"]],
                 
    by = ["tunnel", "bridge"],
    
    aggfunc = ("sum", "mean"),
    
    )

joinet = min_sjoin(dissolvet, bufrede_punkter[["geometry"]])
joinet

## buffer, dissolve og til singlepart (explode)

For å slippe å styre med index, reparering av geometri og lignende hver gang, kan det være greit med en funksjon som gjør akkurat det man vil.

Kaller den buffdissexp for å gjøre det tydelig og eksplisitt hva den gjør (buffer, dissolve og explode):

In [ ]:
def buffdissexp(gdf, avstand, resolution=50, **qwargs):

    kopi = gdf.copy()
        
    kopi["geometry"] = kopi.buffer(avstand, resolution=resolution).make_valid() #reparerer geometrien

    dissolvet = kopi.dissolve(**qwargs).reset_index()
    dissolvet = dissolvet.loc[:, ~dissolvet.columns.str.contains('index|level_')]

    dissolvet.columns = ["_".join(kolonne).strip("_") if isinstance(kolonne, tuple) else kolonne for kolonne in dissolvet.columns]

    singlepart = dissolvet.explode(ignore_index=True)

    return singlepart

Og så slenger jeg på en funksjon som tetter hull i polygoner:

In [ ]:
def tett_hull(gdf):

    # shapely er pakken som håndterer GIS-biten i geopandas. Det vil si, shapely er mellomleddet mellom geopandas og GEOS (GIS-algoritmene som brukes i QGIS, POSTGIS, sf i R osv.)
    from shapely import polygons, get_exterior_ring, get_parts
    from shapely.ops import unary_union
    
    def tett_radvis(x):
        hull_tettet = polygons(get_exterior_ring(get_parts(x)))
        return unary_union(hull_tettet)
    
    kopi = gdf.copy(deep=True)
    kopi["geometry"] = kopi.geometry.map(tett_radvis)

    return kopi

Nå kan man enklere gjenta bufring, dissolving og singleparting. Og hvis man vil endre noe på koden, treger man bare å gjøre det ett sted (i funksjonene man lager).

In [ ]:
buffutinn = buff(bufrede_punkter, 25)
buffutinn = buff(bufrede_punkter, 25)
buffutinn = buff(bufrede_punkter, 25)
buffutinn = buff(bufrede_punkter, 25)
buffutinn = buffdissexp(bufrede_punkter, 12.5)
buffutinn = buffdissexp(bufrede_punkter, -12.5)
buffutinn = tett_hull(bufrede_punkter)
buffutinn = buffdissexp(bufrede_punkter, 1)
buffutinn = buffdissexp(bufrede_punkter, -1)

## Mer om tett_hull()

En mer fleksibel variant ligger her:

In [ ]:
import geopandas as gpd
import sys
sys.path.append(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\Dokumenter\GitHub")

import geopandasgreier as gg # https://github.com/mortewle/geopandasgreier
gg.tett_hull

Velger først ut nabokommunene til Hol og Hemsedal, som inneholder to hull (Hol og Hemsedal).

In [ ]:
import kommfylk # https://github.com/statisticsnorway/kommfylk/blob/main/eksempler/bruk.ipynb
naboer_hol = kommfylk.nabokommuner("3044", aar=2022)
naboer_hemsedal = kommfylk.nabokommuner("3042", aar=2022)
naboer_hol_og_hemsedal = kommuner[kommuner.KOMMUNENR.isin(naboer_hol+naboer_hemsedal)].dissolve()
naboer_hol_og_hemsedal.plot()

Her kan man tette hull under en viss størrelse:

In [ ]:
gg.tett_hull(naboer_hol_og_hemsedal, max_km2 = 1000).plot()

Eller alle hull:

In [ ]:
gg.tett_hull(naboer_hol_og_hemsedal).plot()

## Mer om dissolve

Hvis vi vil ha ulike og/eller flere aggregeringsfunksjoner, kan vi gjøre sånn:

In [ ]:
dissolvet = bufrede_punkter.dissolve(
    
    by = ["bridge", "tunnel"], 
    
    # aggregerings i ordbok (dictionary)
    aggfunc = {"municipality": "first",
               "roadtype": "count", 
               "speed_limit": ["sum", "mean"],
               "traffic_per_day": ["median", np.std],
              }
)

dissolvet = dissolvet.reset_index()

dissolvet

Man kan også bruke egendefinerte aggregeringsfunksjoner.

Enten noe man definerer som en vanlig funksjon, eller en anonym funksjon (lambda) som kan plasseres på selve linja. I dette eksemplet er de to funksjonene like.

In [ ]:
def min_aggfunc(x):
    return x.size - x.count()
    
dissolvet = bufrede_punkter.dissolve(
    
    by = ["bridge", "tunnel"], 
    
    # aggregerings i ordbok (dictionary)
    aggfunc = {"municipality": "first",
               "roadtype": ["first", "count"], 
               "speed_limit": min_aggfunc, 
               "traffic_per_day": lambda x: x.size - x.count(), 
              }
)

dissolvet = dissolvet.reset_index()

dissolvet

Når man dissolver 'by' flere kolonner, får man dumme kolonnenavn i tuple-format, f.eks. (municipality, first):

In [ ]:
dissolvet

Jeg vil ha kolonnene som tekst med underscore, altså municipality_first.

Man kan endre dette med for-loop:

In [ ]:
for kolonne in dissolvet.columns:
    if isinstance(kolonne, tuple):
        dissolvet = dissolvet.rename(columns = {kolonne: "_".join(kolonne).strip("_")})

dissolvet

Eller med list comprehension (raskere enn for-loop):

In [ ]:
dissolvet.columns = ["_".join(kolonne).strip("_") if isinstance(kolonne, tuple) else kolonne for kolonne in dissolvet.columns]
dissolvet

Som er det som gjøres i diss-funksjonen ovenfor:

In [ ]:
diss(bufrede_punkter,
     
     by = ["bridge", "tunnel"],
     
     aggfunc = {"municipality": "first", 
                "speed_limit": ["sum","mean"],
                "traffic_per_day": ["sum","mean"]
                }
)

Det som skjer:

In [ ]:
en_tuple = ("hei", "du", "der", "")
en_tuple

In [ ]:
tuple_samlet_som_string = "_".join(en_tuple)
tuple_samlet_som_string

In [ ]:
uten_underscore_ytterst = tuple_samlet_som_string.strip("_")
uten_underscore_ytterst

## Bruk av ordbok

Ordbøker kan iblant være veldig nyttig. 

For eksempel hvis man har kommunenummer og vil ha navn. Da kan man lage seg en ordbok med navn og nummer.

In [ ]:
from kommfylk import kommuner_fra_api

komm = kommuner_fra_api(2022, navn=True)
komm

In [ ]:
kommuneordbok = {kommnr: kommnavn for kommnr, kommnavn in zip(komm["KOMMUNENR"], komm["NAVN"])}

veger["kommnavn"] = veger["municipality"].map(kommuneordbok)
veger["kommnavn"]

pandas map godtar ordbøker, pandas-kolonner og funksjoner (for funksjoner er det vanligere å bruke apply). Kjekt for radvis manipulering av data. 

Men map/apply er treigere enn å gjøre ting direkte på kolonnen, hvis det er mulig. Det gjelder enklere greier som dette:

In [ ]:
# disse tre linjene gjør det samme
veger["speed_limit_miles"] = veger["speed_limit"].map(lambda x: x*0.62) # treigt
veger["speed_limit_miles"] = veger["speed_limit"].apply(lambda x: x*0.62) # treigt
veger["speed_limit_miles"] = veger["speed_limit"] * 0.62 # raskt

Annet eksempel: bruke ordbok for å lagre lister over kommunenes naboer, hvis man skulle trenge data for området rundt kommunen man beregner for.

In [ ]:
# først klargjør kommunedata og vegdata
kommuner = gpd.read_file(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\data\Basisdata_0000_Norge_25833_Kommuner_FGDB.gdb", 
                         layer="kommune")
kommuner["KOMMUNENR"] = kommuner.kommunenummer
kommuner = kommuner.sort_values("KOMMUNENR")
veger["KOMMUNENR"] = veger.municipality

In [ ]:
# lager en tom ordbok
naboordbok = {}

# looper for hver kommune
for kommune in kommuner.KOMMUNENR.unique():

    #spatial join mellom kommunen og alle andre andre kommuner
    kommunen = kommuner.loc[kommuner.KOMMUNENR == kommune, ["geometry"]]
    andre_kommuner = kommuner.loc[kommuner.KOMMUNENR != kommune, ["KOMMUNENR", "geometry"]]
    joinet = kommunen.sjoin(andre_kommuner)

    # gjør liste over unike kommunenumre til value og kommunen til key i naboordboka
    naboer = list(joinet.KOMMUNENR.unique())
    naboordbok[kommune] = naboer

# nå er kommunene keys, og naboene values. Så dette gir en liste over oslos nabokommuner:
print("Oslos nabokommuner:")
naboordbok["0301"]

Hvis man trenger data for nabokommuner, kan man loope via naboordboka:

In [ ]:
for kommnr, naboer in naboordbok.items(): # items() er en metode for dictionary-typen, som gir oss nøkler og verdier samtidig.
    
    relevante_veger = veger.loc[(veger["KOMMUNENR"]==kommnr) | (veger["KOMMUNENR"].isin(naboer))]

## Enda litt mer om dissolve (dictionary comprehension)
Hvis man har veldig mange kolonner og vil ha 'sum' hvis numerisk og 'first' ellers:

In [ ]:
# dictionary comprehension hvor kolonnenavnet er key og aggregeringsfunksjonen er value
aggfunc = {kolonne: "sum" 
           if ('float' or 'int') in str(bufrede_punkter[kolonne].dtype)
           else 'first'
           for kolonne in bufrede_punkter.columns}
aggfunc

Dictionary comprehension fungerer på samme måte som list comprehension, bare at man spesifiserer nøkkel: verdi (key: value), med kolon.

Man må også fjerne kolonnene som dissolves 'by' og geometri-kolonnen fra ordboka:

In [ ]:
by = ["bridge", "tunnel"]

aggfunc = {kolonne: "sum" if ('float' or 'int') in str(bufrede_punkter[kolonne].dtype) else "first" for kolonne in bufrede_punkter.columns # denne linja er samme som alt over
           if kolonne!="geometry" and kolonne not in by}

aggfunc

Så man man dissolve:

In [ ]:
dissolvet = diss(bufrede_punkter,

                 by = by,
                
                 aggfunc = aggfunc
)
dissolvet